In [12]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import pandas as pd
import fuzzywuzzy
import openai
from fuzzywuzzy import fuzz, process

In [2]:
df = pd.read_csv("C:\Stack overflow\output.csv")

In [3]:
alternate = []
for i in df["Alternate Names"]:
    alternate.append(i.lower())
filtered = alternate
for i in filtered:
    if "?" in i:
        filtered.remove(i)

In [7]:
nlp = spacy.load("en_core_web_trf")
ruler = nlp.add_pipe("entity_ruler", before="ner")

In [8]:
pattern = {"pattern": [{"TEXT": {"FUZZY": {"IN": ["palani", "venao"] }}}], "label": "Location"}
pattern1 = [{"TEXT": {"IN": filtered}}]
ruler.add_patterns([pattern])

In [9]:
doc = nlp("I am going to palan next week")

In [10]:
def checking_gpe(doc):
    place = []
    dict = []
    real = []
    num = 0
    #Checking the tokens in database 
    for token in doc:
        for pattern in pattern1:
            if token.text.lower() in pattern["TEXT"]["IN"]:
                place.append(token)
                print(f"Exact location found!!!\nLocation : {token.text} \nCanonical name : {token.text.upper()}\nConfidence score : 100")
        dict.append(process.extractOne(token.text, alternate, scorer= fuzz.ratio))
        real.append(token.text)
    #If token not found in database then check if there is any spelling errors
    if len(place) == 0:
        for x,y in dict:
            num += 1
            if y > 80:
                print(f"Spelling error dectected!!!\nError word : {real[num - 1]} \nCanonical name : {x.upper()}\nConfidence score : {y}")
                break
    #If token not found in both database and spelling error in token then proceeds for contexual analysis
    

In [11]:
checking_gpe(doc)

Spelling error dectected!!!
Error word : palan 
Canonical name : PALANI
Confidence score : 91
